In [1]:
import pandas as pd

df = pd.read_csv('new_classification_0809.csv')
df.head()

,Unnamed: 0,조_항목,항목,원본,조항구분,조항구분_예측,파일명
0,0,목적 및 대상,1조 1항,본 계약은 음식물류 폐기물의 자원화를 효율적으로 추진하기 위하여 필요한 사항을 약정...,['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
1,1,목적 및 대상,1조 2항,위탁처리대상은 [지역명1]군에서 분리 수거한 음식물류폐기물로 한다.,['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
2,2,위탁업무의 내용 및 처리방법,2조 1항,"음식물폐기물의 수집․운반은 “갑”이하고 수집,운반된 음식물류 폐기물의 처리는 “을”...",['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
3,3,위탁업무의 내용 및 처리방법,2조 2항,"“을”은 “갑”이 수집,운반한 음식물류폐기물을 사업장에서 관련 법령에 적법하게 ...",['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
4,4,계약기간,3조 1항,위수탁기간은 [기간]로 한다.,['필수조항1'],['필수조항1'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx


In [16]:
cnt = 0
for i in range(len(df)):
    if df.iloc[i]['조항구분'] == df.iloc[i]['조항구분_예측']:
        cnt += 1
print(f"Accuracy = {cnt / len(df) * 100:.4f}")

Accuracy = 44.1317


## 정답이 후보군에 포함되어 있는 비율 파악

In [2]:
# 1. 각 정답 조항구분 별로 나타나는 조항구분_예측의 구성 형태 파악
# 2. 정답이 후보군에 포함되어 있는지 확인
import re

def evaluate_total(df):
    total = len(df)
    print(f"전체 데이터 수: {total}개")
    cnt = 0
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        # 'nan' 또는 '필수조항1' 또는 '필수조항1', '필수조항2'
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if set(labels).issubset(set(preds)):
            cnt += 1
    print(f"후보군이 정답을 포함하고 있는 항 수: {cnt}개")
    return cnt / total * 100


In [3]:
evaluate_total(df)

전체 데이터 수: 37123개
후보군이 정답을 포함하고 있는 항 수: 23656개


63.72329822482019

In [4]:
# 1. 각 정답 조항구분 별로 나타나는 조항구분_예측의 구성 형태 파악
# 2. 정답이 후보군에 포함되어 있는지 확인
import re

def evaluate(df):
    total = 0
    cnt = 0
    for i in range(len(df)):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        # 'nan' 또는 '필수조항1' 또는 '필수조항1', '필수조항2'
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        if ['nan'] == labels:
            continue
        if 'nan' in labels:
            labels.remove('nan')
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if set(labels).issubset(set(preds)):
            cnt += 1
        total += 1
    print(f"전체 데이터 수: {total}개")
    print(f"후보군이 정답을 포함하고 있는 항 수: {cnt}개")
    return cnt / total * 100


In [5]:
evaluate(df)

전체 데이터 수: 16585개
후보군이 정답을 포함하고 있는 항 수: 11586개


69.8583056979198

## 정답 조항의 개수에 따른 조항구분 예측 후보의 개수 분포

In [6]:
from collections import defaultdict

def number_of_cands(df):
    cnt_dict = defaultdict(dict)
    total = len(df)
    print(f"전체 데이터 수: {total}개")
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        # 'nan'인 경우 0개로 취급
        if ['nan'] == labels:
            if ['nan'] == preds:
                if 0 not in cnt_dict[0]:
                    cnt_dict[0][0] = 1
                else:
                    cnt_dict[0][0] += 1
            else:
                if len(preds) not in cnt_dict[0]:
                    cnt_dict[0][len(preds)] = 1
                else:
                    cnt_dict[0][len(preds)] += 1
        elif 'nan' in labels:
            labels.remove('nan')
            if ['nan'] == preds:
                if 0 not in cnt_dict[len(labels)]:
                    cnt_dict[len(labels)][0] = 1
                else:
                    cnt_dict[len(labels)][0] += 1
            else:
                if len(preds) not in cnt_dict[len(labels)]:
                    cnt_dict[len(labels)][len(preds)] = 1
                else:
                    cnt_dict[len(labels)][len(preds)] += 1
        else:
            if len(preds) not in cnt_dict[len(labels)]:
                cnt_dict[len(labels)][len(preds)] = 1
            else:
                cnt_dict[len(labels)][len(preds)] += 1
                
    # sorting
    sorted_cnt_dict = {}
    sorted_by_num_cnt_dict = {}
    
    for k, v_dict in sorted(cnt_dict.items()):
        sorted_cnt_dict[k] = {a:b for a, b in sorted(v_dict.items(), key=lambda x: x[1], reverse=True)}
        sorted_by_num_cnt_dict[k] = {a:b for a, b in sorted(v_dict.items())}

    return sorted_cnt_dict, sorted_by_num_cnt_dict

In [7]:
sorted_cnt_dict, sorted_by_num_cnt_dict = number_of_cands(df)

전체 데이터 수: 37123개


In [8]:
sorted_cnt_dict

{0: {0: 12070,
  1: 5115,
  2: 2074,
  3: 777,
  4: 252,
  5: 151,
  6: 62,
  7: 34,
  9: 2,
  8: 1},
 1: {1: 7574,
  2: 4869,
  3: 1914,
  4: 776,
  5: 286,
  6: 103,
  7: 71,
  9: 14,
  8: 12,
  10: 2},
 2: {2: 197, 3: 167, 4: 126, 5: 72, 1: 71, 6: 28, 8: 26, 7: 9, 12: 1, 11: 1},
 3: {5: 50, 4: 49, 3: 48, 2: 31, 6: 14, 1: 13, 8: 8, 7: 7, 9: 2},
 4: {5: 10, 4: 10, 3: 6, 2: 3, 6: 3, 7: 1, 10: 1, 1: 1, 8: 1, 9: 1},
 5: {8: 2, 7: 2, 4: 1},
 6: {4: 1, 3: 1}}

In [9]:
sorted_by_num_cnt_dict

{0: {0: 12070,
  1: 5115,
  2: 2074,
  3: 777,
  4: 252,
  5: 151,
  6: 62,
  7: 34,
  8: 1,
  9: 2},
 1: {1: 7574,
  2: 4869,
  3: 1914,
  4: 776,
  5: 286,
  6: 103,
  7: 71,
  8: 12,
  9: 14,
  10: 2},
 2: {1: 71, 2: 197, 3: 167, 4: 126, 5: 72, 6: 28, 7: 9, 8: 26, 11: 1, 12: 1},
 3: {1: 13, 2: 31, 3: 48, 4: 49, 5: 50, 6: 14, 7: 7, 8: 8, 9: 2},
 4: {1: 1, 2: 3, 3: 6, 4: 10, 5: 10, 6: 3, 7: 1, 8: 1, 9: 1, 10: 1},
 5: {4: 1, 7: 2, 8: 2},
 6: {3: 1, 4: 1}}

## 정답 조항의 종류에 따른 예측 조항의 분포


In [10]:
from collections import Counter

print(f"전체 데이터 수: {len(df)}")

def predicted_form(df, trg_label):
    counter = Counter()
    cnt = 0
    total = len(df)
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if trg_label in labels:
            counter.update(preds)
            cnt += 1
            
    return cnt, counter

전체 데이터 수: 37123


In [11]:
cnt, counter = predicted_form(df, '필수조항1')
print(f"필수조항1이 나오는 횟수는 {cnt}")
print(counter)

필수조항1이 나오는 횟수는 880
Counter({'필수조항1': 830, '옵션조항3': 167, '필수조항3': 151, '필수조항2': 128, '필수조항9': 61, '필수조항10': 60, '필수조항8': 38, 'nan': 29, '옵션조항2': 25, '필수조항6': 20, '필수조항12': 18, '필수조항7': 13, '필수조항5': 10, '필수조항4': 10, '옵션조항5': 8, '필수조항13': 8, '옵션조항8': 6, '필수조항11': 4, '옵션조항7': 3, '옵션조항1': 3})


In [12]:
cnt, counter = predicted_form(df, '필수조항2')
print(f"필수조항2이 나오는 횟수는 {cnt}")
print(counter)

필수조항2이 나오는 횟수는 162
Counter({'필수조항1': 132, '필수조항3': 119, '필수조항2': 118, 'nan': 20, '필수조항9': 20, '필수조항8': 18, '옵션조항3': 10, '옵션조항2': 6, '필수조항4': 4, '필수조항10': 3, '필수조항7': 3, '옵션조항1': 2, '필수조항13': 2, '필수조항5': 2, '옵션조항5': 1, '필수조항12': 1})


### 각 조항별로 예측이 되는 비율 구하기 - 키워드 추가 필요한 조항(?)

In [13]:
label_list = ['필수조항1', '필수조항2', '필수조항3', '필수조항4', '필수조항5', '필수조항6', '필수조항7',
       '필수조항8', '필수조항9', '필수조항10', '필수조항10.1', 
    '필수조항11', '필수조항12', '필수조항13', '옵션조항1', '옵션조항2', '옵션조항3', '옵션조항4', '옵션조항5', 
            '옵션조항6', '옵션조항7', '옵션조항8', '옵션조항9', '옵션조항10', '옵션조항12']

In [14]:
for trg_label in label_list:
    cnt, counter = predicted_form(df, trg_label)
    print(f"{trg_label:>8} --> 정답으로 존재하는 개수: {cnt:4d}개, 예측후보에 존재하는 개수: {counter[trg_label]:4d}개, 비율: {counter[trg_label] / cnt * 100:.4f}%")

   필수조항1 --> 정답으로 존재하는 개수:  880개, 예측후보에 존재하는 개수:  830개, 비율: 94.3182%
   필수조항2 --> 정답으로 존재하는 개수:  162개, 예측후보에 존재하는 개수:  118개, 비율: 72.8395%
   필수조항3 --> 정답으로 존재하는 개수:  795개, 예측후보에 존재하는 개수:  694개, 비율: 87.2956%
   필수조항4 --> 정답으로 존재하는 개수:  895개, 예측후보에 존재하는 개수:  731개, 비율: 81.6760%
   필수조항5 --> 정답으로 존재하는 개수:  869개, 예측후보에 존재하는 개수:  697개, 비율: 80.2071%
   필수조항6 --> 정답으로 존재하는 개수: 1088개, 예측후보에 존재하는 개수:  889개, 비율: 81.7096%
   필수조항7 --> 정답으로 존재하는 개수:  863개, 예측후보에 존재하는 개수:  623개, 비율: 72.1900%
   필수조항8 --> 정답으로 존재하는 개수: 1033개, 예측후보에 존재하는 개수:  781개, 비율: 75.6050%
   필수조항9 --> 정답으로 존재하는 개수:  924개, 예측후보에 존재하는 개수:  822개, 비율: 88.9610%
  필수조항10 --> 정답으로 존재하는 개수:  226개, 예측후보에 존재하는 개수:   93개, 비율: 41.1504%
필수조항10.1 --> 정답으로 존재하는 개수:  301개, 예측후보에 존재하는 개수:    0개, 비율: 0.0000%
  필수조항11 --> 정답으로 존재하는 개수: 2724개, 예측후보에 존재하는 개수: 1384개, 비율: 50.8076%
  필수조항12 --> 정답으로 존재하는 개수: 1562개, 예측후보에 존재하는 개수: 1429개, 비율: 91.4853%
  필수조항13 --> 정답으로 존재하는 개수:  263개, 예측후보에 존재하는 개수:  209개, 비율: 79.4677%
   옵션조항1 --> 정답으로 존재하는 개수: 1116개, 예